## Get Reviews from Steam in JSON format for local data processing 

This notebook contains an abstracted version of the code used in the course of the master thesis for the extraction of the review data. 


### Get all Page cursors

Since Steam handles there JSON-pages via cursors consisting of random characters, you need to get all the cursors in order to iterate thru all the Reviews of a game (if it has over 100 reviews).

In [ ]:
import requests
import json
import os.path
import urllib.parse


cursorlist = [] #empty list for retrieved cursors

reviewurl = '' #enter here the url to the first json-page of the reviews; but leave the cursor= parameter empty!


#Opens the next json-page when the cursor-parameter is passed and call sthe get_cursor function to get the next cursor

def get_jsonpages (cursor): # needs a cursor value to be initated; 

    if cursor == '*':
       url = reviewurl + cursor
    else:
       save_cursor = urllib.parse.quote(str(cursor))  #urlencoding of the cursor 
       url = reviewurl + save_cursor
       
    response = requests.get(url) 
    data = response.json() 
    
    if data['query_summary']['num_reviews'] == 0: #checks reviewamount; if no reviews are in the page; the function stops
        return
    else:
        return get_cursor(data)  #calls the get_cursor functions, which retrieves the cursor from the json 


#Retrieves the cursor from the JSON and calls the get_jsonpages function to open the next dataset 

def get_cursor(response:dict):
    
    cursor = response['cursor']
    cursorlist.append(cursor)

    if response['query_summary']['num_reviews'] == 0:
        return
    else:
        return get_jsonpages(cursor)

get_jsonpages('*') # starts the functions 
cursorlist.insert(0, '*') # inserts the missing default value '*' in the list for the next step



### Save Files to Local Directory

Steamworks limits the API-Calls to 100.000 calls a day. If you are looking at a huge amount of reviews, you might as well save them to a local directory and process them afterwards. 

In [ ]:
# Iteration through the cursorlist
for c in cursorlist:
    if c == '*':
       url = cursorurl + c
    else:
       save_cursor = urllib.parse.quote(str(c)) #urlencoding of the cursors in the list 
       url = cursorurl + save_cursor
    
    index = cursorlist.index(c) #index for filenames 
    filename = f'json_page{index}.json'
    resp = requests.get(url)
    jsondata= resp.json()
    
    path= '' # enter the path you want to save the files to here 
    saveToFolder = os.path.join (path,filename) # joins path and filename 
    
    #opens the path and saves the file; if the file already exists it will be overwritten 
    with open(saveToFolder, 'w') as fp:  
        json.dump(jsondata, fp, indent=4)
